In [86]:
import json
import pandas as pd

In [87]:
file_path = './data/student_depression_dataset.csv'
mental_healt_df = pd.read_csv(file_path)

In [88]:
mental_healt_df.shape

(27901, 18)

In [89]:
mental_healt_df['Depression'].value_counts()

Depression
1    16336
0    11565
Name: count, dtype: int64

In [90]:
charts_info_dict = {}
depression_count = mental_healt_df['Depression'].value_counts()

charts_info_dict['depression-rates'] = {
    'Positius': str(depression_count[1]),
    'Negatius': str(depression_count[0])
}

suicide_thoughts_count = mental_healt_df['Have you ever had suicidal thoughts ?'].value_counts()
charts_info_dict['suicidal-thougths-rates'] = {
    'Positius': str(suicide_thoughts_count['Yes']),
    'Negatius': str(suicide_thoughts_count['No'])
}

In [91]:
city_counts = mental_healt_df['City'].value_counts()
city_counts = city_counts[city_counts >= 400].index
mental_healt_df = mental_healt_df[mental_healt_df['City'].isin(city_counts)]

In [ ]:
city_depression = mental_healt_df.groupby(['City', 'Depression']).count()['id']

city_totals = city_depression.groupby(level=0).sum()
ordered_cities = city_totals.sort_values(ascending=False).index

charts_info_dict['city-depression'] = {}

for city in ordered_cities:
    city_data = city_depression.loc[city]
    
    if not isinstance(city_data, pd.Series):
        city_data = pd.Series({city_data.index: city_data})

    charts_info_dict['city-depression'][city] = {}

    for depression in [1, 0]:
        if depression in city_data:
            depression_key = 'Positiu' if depression == 1 else 'Negatiu'
            charts_info_dict['city-depression'][city][depression_key] = str(city_data[depression])


In [93]:
output_json_path = '../public/data/mental-health-data.json'
with open(output_json_path, 'w', encoding='utf-8') as file:
    json.dump(charts_info_dict, file, ensure_ascii=False, indent=4)